In [1]:
import pandas as pd
import numpy as np

In [2]:
averaged_car_measurements = pd.read_csv('data/averaged_car_measurements.csv')

# Change date to datetime, format is %Y-%m-%d
averaged_car_measurements['date'] = pd.to_datetime(averaged_car_measurements['date'], format='%Y-%m-%d')

averaged_car_measurements.head()

,Hour,date,NO_ugm3,NO2_ugm3,O3_ugm3,CO_mgm3,CO2_mgm3,PM25_ugm3,SiteID,Lat,Long,month,day_of_week
0,8,2021-06-28,12.044255,0.085176,39.303789,0.351863,878.897362,5.072396,1,53.348754,-6.257607,6,0
1,9,2021-06-28,5.263025,9.600681,29.413390,0.350409,842.139168,7.998587,1,53.348754,-6.257607,6,0
2,9,2021-08-16,10.318074,3.006013,61.380364,0.380456,809.073556,6.122852,1,53.348754,-6.257607,8,0
3,9,2021-10-22,456.213318,NaN,NaN,0.432339,850.479574,5.752877,1,53.348754,-6.257607,10,4
4,9,2022-07-29,15.093107,NaN,NaN,0.415959,805.524520,4.757151,1,53.348754,-6.257607,7,4


In [3]:
# Prep hourly_weather data
hourly_weather = pd.read_csv('Weather-Air_pollution/merged_weather_data.csv')

# Convert hour to integer
hourly_weather['hour'] = hourly_weather['hour'].str.replace(':00', '')
hourly_weather['hour'] = hourly_weather['hour'].astype(int)

# Drop latitute, longitude and datetime columns as they are not needed
hourly_weather = hourly_weather.drop(['latitude', 'longitude','datetime'], axis=1)

# Change date to datetime, format is %d/%m/%Y
hourly_weather['date'] = pd.to_datetime(hourly_weather['date'], format='%d/%m/%Y')

hourly_weather.head()

,date,hour,avgtempC,maxtempC,mintempC,sunHour,uvIndex,humidity,winddirDegree,windspeedKmph,cloudcover,precipMM,pressure
0,2021-05-01,0,6,8,3,13,3,80,305,9,5,0.0,1017
1,2021-05-01,1,6,8,3,13,3,82,306,10,15,0.0,1017
2,2021-05-01,2,6,8,3,13,3,83,306,11,25,0.0,1017
3,2021-05-01,3,6,8,3,13,3,85,307,12,35,0.0,1017
4,2021-05-01,4,6,8,3,13,3,84,308,12,41,0.0,1017


In [54]:
air_pollution_sensors.head()

,DCC-AQ1-co,DCC-AQ1-no,DCC-AQ10-no,DCC-AQ13-no,DCC-AQ5-no,DCC-AQ6-no,DCC-AQ1-no2,DCC-AQ10-no2,DCC-AQ13-no2,DCC-AQ17-no2,...,TNO4325-tsp,TNO4390-tsp,TNO4435-tsp,TNO4437-tsp,TNO4438-tsp,TNO4488-tsp,TNT1088-tsp,TNT1296-tsp,hour,date
0,0.47,6.54,43.11,27.77,0.46,13.82,68.00,63.11,-1.47,38.30,...,12.85,NaN,16.67,NaN,13.75,14.42,5.73,NaN,22,2021-04-30
1,0.55,8.21,26.41,46.69,8.96,8.39,60.76,58.34,2.92,43.54,...,14.57,NaN,19.75,NaN,39.72,17.35,7.63,NaN,23,2021-04-30
2,0.49,2.19,18.79,43.65,0.76,3.42,52.77,60.68,3.44,35.56,...,14.05,NaN,17.40,NaN,18.57,14.80,9.20,NaN,0,2021-05-01
3,0.47,5.92,13.31,34.25,2.09,1.65,51.68,50.40,-0.89,21.63,...,15.40,NaN,10.92,NaN,17.20,11.80,4.78,NaN,1,2021-05-01
4,0.39,2.32,6.11,35.84,0.68,0.80,42.85,32.74,2.60,29.98,...,13.35,NaN,12.33,NaN,12.35,6.73,3.75,NaN,2,2021-05-01


In [55]:
# Prepare Air pollution sensor data
air_pollution_sensors = pd.read_csv('dublin_official_measurements/dublin_official_measurements_sonitus_api.csv')

# Extract hour and date from datetime column
air_pollution_sensors['hour'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.hour
air_pollution_sensors['date'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.date

# Drop datetime column
air_pollution_sensors = air_pollution_sensors.drop(['datetime'], axis=1)

#Drop longitude and latitude columns
#air_pollution_sensors = air_pollution_sensors.drop(['longitude', 'latitude'], axis=1)

# Pivot the data for each sensor, indicated by the column 'serial_number', keeping the date and hour columns and resetting the index
air_pollution_sensors2 = air_pollution_sensors.pivot_table(index=['date', 'hour'], columns='serial_number').reset_index()

# flatten the column index
air_pollution_sensors2.columns = ['_'.join(col).strip() for col in air_pollution_sensors2.columns.values]

#rename date_ and hour_ columns
air_pollution_sensors2.rename(columns={'date_': 'date', 'hour_': 'hour'}, inplace=True)

#Convert the date column to datetime
air_pollution_sensors2['date'] = pd.to_datetime(air_pollution_sensors2['date'])

#convert the date column to datetime

air_pollution_sensors2.head()

KeyError: 'serial_number'

Weather data

In [62]:
# Left join the two dataframes on hour and date together
Full_data = pd.merge(averaged_car_measurements, hourly_weather, how='left', left_on=['date','Hour'], right_on=['date', 'hour'])
Full_data.head()
# Left join the air pollution sensors data
air_pollution_sensors = pd.read_csv('dublin_official_measurements/dublin_official_measurements_sonitus_api.csv')
# Extract hour and date from datetime column
air_pollution_sensors['hour'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.hour
air_pollution_sensors['date'] = pd.to_datetime(air_pollution_sensors['datetime']).dt.date
# Drop datetime column
air_pollution_sensors = air_pollution_sensors.drop(['datetime'], axis=1)
#convert the date column to datetime
air_pollution_sensors['date'] = pd.to_datetime(air_pollution_sensors['date'])

Full_data = pd.merge(Full_data, air_pollution_sensors, how='left', left_on=['date','Hour'], right_on=['date', 'hour'])

#Drop columns with more than 50% of missing values
Full_data = Full_data.dropna(thresh=0.5*len(Full_data), axis=1)
# Drop hour column
Full_data = Full_data.drop(['Hour'], axis=1)

NO_ugm3         0.053550
NO2_ugm3        0.182880
O3_ugm3         0.184748
CO_mgm3         0.018769
CO2_mgm3        0.029224
                  ...   
DCC-AQ22-so2    0.036740
TNO2161-tsp     0.045919
TNO2162-tsp     0.134046
TNO4435-tsp     0.317723
TNT1088-tsp     0.024190
Length: 74, dtype: float64


In [64]:
#Change full data to be sorted by date
#import full data again
Full_data = Full_data.sort_values(by='date')

# Export to csv
Full_data.to_csv('data/Full_data.csv', index=False)